In [1]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as k

Using TensorFlow backend.


In [2]:
class LeNet:
    @staticmethod
    def build(numChannels, imgRows, imgCols, numClasses, activation="relu", weightsPath=None):
        # sequential initialize the mode
        model = Sequential()
        inputShape = (imgRows, imgCols, numChannels)
        
        #if we are using "last channels", update the input shape
        
        inputShape = (imgRows, imgCols, numChannels)
        # define the first set of CONV => ACTIVATION => POOL layers
        model.add(Conv2D(32,5,padding="Same",input_shape=inputShape)) #20=convolution Filter,5=each filter size 5*5
        model.add(Activation(activation))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        # define the second set of CONV => ACTIVATION => POOL layers
        model.add(Conv2D(32,5,padding="Same"))
        model.add(Activation(activation))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        # define the first FC => ACTIVATION layers
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation(activation))
        # define the second FC layer
        model.add(Dense(numClasses))
        # lastly, define the soft-max classifier
        model.add(Activation("softmax"))
        # if a weights path is supplied (inicating that the model was pre-trained), then load the weights
        # return the construction architecture
        return model
    

In [3]:
from keras.optimizers import SGD
opt = SGD(lr=0.01)
model = LeNet.build(numChannels=3, imgRows=227, imgCols=227, numClasses=2)
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 227, 227, 32)      2432      
_________________________________________________________________
activation_1 (Activation)    (None, 227, 227, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 113, 113, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 113, 113, 32)      25632     
_________________________________________________________________
activation_2 (Activation)    (None, 113, 113, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)           

In [38]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator



train_datagen = ImageDataGenerator(rescale=1./225,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')


test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

model.fit_generator(training_set,samples_per_epoch = 8000,nb_epoch = 10,validation_data = test_set,nb_val_samples = 2000)




Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


/home/marzan/work_env/local/lib/python3.5/site-packages/ipykernel_launcher.py:22: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/marzan/work_env/local/lib/python3.5/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=10, validation_steps=2000, validation_data=<keras.pre..., steps_per_epoch=250)`


Epoch 1/10
250/250 [==============================] - 549s 2s/step - loss: 0.6929 - accuracy: 0.5424 - val_loss: 0.7294 - val_accuracy: 0.5775
Epoch 2/10
250/250 [==============================] - 543s 2s/step - loss: 0.6682 - accuracy: 0.5861 - val_loss: 0.6334 - val_accuracy: 0.6367
Epoch 3/10
250/250 [==============================] - 542s 2s/step - loss: 0.6509 - accuracy: 0.6190 - val_loss: 0.6025 - val_accuracy: 0.6647
Epoch 4/10
250/250 [==============================] - 541s 2s/step - loss: 0.6251 - accuracy: 0.6564 - val_loss: 0.5981 - val_accuracy: 0.6840
Epoch 5/10
250/250 [==============================] - 544s 2s/step - loss: 0.5962 - accuracy: 0.6783 - val_loss: 0.5395 - val_accuracy: 0.7133
Epoch 6/10
250/250 [==============================] - 543s 2s/step - loss: 0.5744 - accuracy: 0.6985 - val_loss: 0.4861 - val_accuracy: 0.7328
Epoch 7/10
250/250 [==============================] - 542s 2s/step - loss: 0.5458 - accuracy: 0.7256 - val_loss: 0.6246 - val_accuracy: 0.7063

In [39]:
# serialize model to JSON
from keras.models import model_from_json
model_json = model.to_json()
with open("model_lenet.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_lenet_weights.h5")
print("Saved model to disk")

Saved model to disk


In [47]:
from keras.models import model_from_json
# Model reconstruction from JSON file
with open('model_lenet.json', 'r') as f:
    model_after = model_from_json(f.read())

# Load weights into the new model
model_after.load_weights('model_lenet_weights.h5')

In [51]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
img = cv2.imread("/media/marzan/mini storage/CNN/cat.jpg")

In [52]:
reshape = cv2.resize(img,(64,64))
dimention_currection = np.reshape(reshape,[1,64,64,3])

In [53]:
predict = model_after.predict(dimention_currection)
predict

array([[1.00000e+00, 4.17571e-35]], dtype=float32)